# CityBikes

Send a request to CityBikes for the city of your choice. 

In [73]:
import requests
import pandas as pd

# Fetch the list of all networks
response = requests.get("http://api.citybik.es/v2/networks")
data = response.json()

# Check if the request was successful (status code 200)
if response.status_code == 200:
    city_name = input("Enter name of city: ")

    filtered_networks = [network for network in data["networks"] if network["location"]["city"].lower() == city_name.lower()]

    if filtered_networks:
        chosen_network = filtered_networks[0]
        chosen_network_id = chosen_network["id"]

        chosen_network_response = requests.get(f"http://api.citybik.es/v2/networks/{chosen_network_id}")
        chosen_network_data = chosen_network_response.json()

        bike_stations = chosen_network_data.get("network", {}).get("stations", [])

        if bike_stations:
            # Create a DataFrame from the bike_stations list
            df = pd.DataFrame(bike_stations)

            print(f"Bike Stations in {city_name}:\n")
            print(df)
        else:
            print(f"No bike stations found for {city_name}.")
    else:
        print(f"No networks found for {city_name}.")
else:
    print(f"Failed to retrieve network data. Status code: {response.status_code}")


Bike Stations in madrid:

     empty_slots                                              extra  \
0             24  {'address': 'Calle Miguel Moya nº 1,', 'light'...   
1             17  {'address': 'Plaza del Conde del Valle de Súch...   
2             16  {'address': 'Calle Hortaleza nº 63,', 'light':...   
3             18  {'address': 'Calle Hortaleza nº 75,', 'light':...   
4             24  {'address': 'Plaza de San Miguel nº 9,', 'ligh...   
..           ...                                                ...   
389            8  {'address': 'Calle del tejo,', 'light': 'yello...   
390            8  {'address': ' Calle Babilonia, 19,', 'light': ...   
391            4  {'address': 'Calle Albelda, 6,', 'light': 'red...   
392           27  {'address': '311 - Calle del Profesor Arangure...   
393            6  {'address': 'Calle Soto Hidalgo, 2,', 'light':...   

     free_bikes                                id   latitude  longitude  \
0             3  82e78d26829a2dc89122e8f0988e2

In [74]:
df.info(), print(type(df))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 394 entries, 0 to 393
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   empty_slots  394 non-null    int64  
 1   extra        394 non-null    object 
 2   free_bikes   394 non-null    int64  
 3   id           394 non-null    object 
 4   latitude     394 non-null    float64
 5   longitude    394 non-null    float64
 6   name         394 non-null    object 
 7   timestamp    394 non-null    object 
dtypes: float64(2), int64(2), object(4)
memory usage: 24.8+ KB
<class 'pandas.core.frame.DataFrame'>


(None, None)

In [75]:
# Notice a column named extra which I dont know it's content.
# Let's explore it.
print(type(df['extra']))


<class 'pandas.core.series.Series'>


Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [76]:
# Create function to retrieve columns of interest
def get_bike_stations_data(city_name):
    filtered_networks = []
    for network in data["networks"]:
        if network["location"]["city"].lower() == city_name.lower():
            filtered_networks.append(network)

    if filtered_networks:
        chosen_network = filtered_networks[0]
        chosen_network_id = chosen_network["id"]

        chosen_network_response = requests.get(f"http://api.citybik.es/v2/networks/{chosen_network_id}")
        chosen_network_data = chosen_network_response.json()

        bike_stations = chosen_network_data["network"]["stations"]

        # Create a list of dictionaries to store station data
        station_data_list = []

        for station in bike_stations:
            latitude = station["latitude"]
            longitude = station["longitude"]
            station_name = station["name"]  # Include station name

            # Check if 'ebikes' is in the 'extra' dictionary
            if 'extra' in station and 'ebikes' in station['extra']:
                total_bikes = station["free_bikes"] + station["empty_slots"] + station['extra']['ebikes']
            else:
                total_bikes = station["free_bikes"] + station["empty_slots"]

            station_data = {
                "Station Name": station_name,
                "Latitude": latitude,
                "Longitude": longitude,
                "Total Bikes": total_bikes
            }

            station_data_list.append(station_data)

        # Create a DataFrame from the list of dictionaries
        df = pd.DataFrame(station_data_list)

        print(f"Bike Stations in {city_name} (DataFrame):\n")
        print(df)

        # Return the DataFrame
        return df
    else:
        print(f"No networks found for {city_name}.")
        # Return an empty DataFrame if no networks are found
        return pd.DataFrame()

# Fetch the list of all networks
response = requests.get("http://api.citybik.es/v2/networks")
data = response.json()

# Take user input for the city name
city_name = city_name

# Call the function to retrieve and display bike station data and save it in the 'city_bike_df' variable
city_bike_df = get_bike_stations_data(city_name)


Bike Stations in madrid (DataFrame):

                              Station Name   Latitude  Longitude  Total Bikes
0                          2 - Miguel Moya  40.420589  -3.705842           27
1                   3 - Plaza Conde Suchil  40.430294  -3.706917           19
2                  6 - Colegio Arquitectos  40.424148  -3.698447           19
3                            7 - Hortaleza  40.425191  -3.697771           19
4                  9 - Plaza de San Miguel  40.415606  -3.709508           24
..                                     ...        ...        ...          ...
389                   272 - Calle del tejo  40.398001  -3.678027           23
390              559 - Calle Babilonia, 19  40.467530  -3.588235           23
391                       528 - Albelda, 6  40.449422  -3.657367           24
392  311 - Calle del Profesor Aranguren, 1  40.448931  -3.730518           27
393                  563 - Soto Hidalgo, 2  40.460647  -3.582075           27

[394 rows x 4 columns]


In [77]:
# Review and compare make up of the dataframe
city_bike_df.shape, city_bike_df.info(), 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 394 entries, 0 to 393
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Station Name  394 non-null    object 
 1   Latitude      394 non-null    float64
 2   Longitude     394 non-null    float64
 3   Total Bikes   394 non-null    int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 12.4+ KB


((394, 4), None)

Put your parsed results into a DataFrame.

In [78]:
# Parsed into a dataframe in the function above


In [79]:
# We then view structure of df and city_bike_df
df.info(), city_bike_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 394 entries, 0 to 393
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   empty_slots  394 non-null    int64  
 1   extra        394 non-null    object 
 2   free_bikes   394 non-null    int64  
 3   id           394 non-null    object 
 4   latitude     394 non-null    float64
 5   longitude    394 non-null    float64
 6   name         394 non-null    object 
 7   timestamp    394 non-null    object 
dtypes: float64(2), int64(2), object(4)
memory usage: 24.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 394 entries, 0 to 393
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Station Name  394 non-null    object 
 1   Latitude      394 non-null    float64
 2   Longitude     394 non-null    float64
 3   Total Bikes   394 non-null    int64  
dtypes: float64(2), int64(1), object(1)
memory usag

(None, None)

##### Next step is to merge the two dataframes together

In [80]:

# Concatenate the two DataFrames along the columns axis (axis=1)
merged_city = pd.concat([df, city_bike_df], axis=1)

# Display the merged DataFrame
merged_city.info() # This gives us same number of rows and 15 columns. Making progress



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 394 entries, 0 to 393
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   empty_slots   394 non-null    int64  
 1   extra         394 non-null    object 
 2   free_bikes    394 non-null    int64  
 3   id            394 non-null    object 
 4   latitude      394 non-null    float64
 5   longitude     394 non-null    float64
 6   name          394 non-null    object 
 7   timestamp     394 non-null    object 
 8   Station Name  394 non-null    object 
 9   Latitude      394 non-null    float64
 10  Longitude     394 non-null    float64
 11  Total Bikes   394 non-null    int64  
dtypes: float64(4), int64(3), object(5)
memory usage: 37.1+ KB


In [81]:
# View merged_city
merged_city.head(2)

,empty_slots,extra,free_bikes,id,latitude,longitude,name,timestamp,Station Name,Latitude,Longitude,Total Bikes
0,24,"{'address': 'Calle Miguel Moya nº 1,', 'light'...",3,82e78d26829a2dc89122e8f0988e2a28,40.420589,-3.705842,2 - Miguel Moya,2023-09-07T00:06:34.970000Z,2 - Miguel Moya,40.420589,-3.705842,27
1,17,{'address': 'Plaza del Conde del Valle de Súch...,2,248a049595da65f6aada65476ca1f958,40.430294,-3.706917,3 - Plaza Conde Suchil,2023-09-07T00:06:35.512000Z,3 - Plaza Conde Suchil,40.430294,-3.706917,19


In [84]:
# Already put in a dataframe in the function above.
print(type(merged_city))

<class 'pandas.core.frame.DataFrame'>


In [85]:
merged_city.to_csv(r'C:\Users\affuy\Documents\Data_Sets\df_2.csv')
